In [1]:
# !pip install selenium pandas

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/ea/37/d07ed9d13e571b2115d4ed6956d156c66816ceec0b03b2e463e80d09f572/selenium-4.32.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/69/8e/3f6dfda475ecd940e786defe6df6c500734e686c9cd0a0f8ef6821e9b2f2/trio-0.30.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/c7/19/eb640a397bba49ba49ef9dbe2e7e5c04202ba045b6ce2ec36e9cadc51e04/trio_websocket-0.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb7537a3/websocket_client-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/77/06/bb80f5f86020c4551da315d78b3ab75e8228f89f0162f2c3a819e


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import numpy as np

In [3]:
import time
time.sleep(1.5)

> Lấy mã tất cả công ty lọc Sàn chứng khoán theo API

In [4]:
def get_data_from_page(skip):
    url = f"https://cafef.vn/du-lieu/ajax/pagenew/databusiness/congtyniemyet.ashx?centerid=0&skip={skip}&take=20&major=0"
    response = requests.get(url)

    symbols = []
    if response.status_code == 200:
        try:
            data = response.json()
            for company in data.get('Data', []):
                if company.get('TradeCenterId') == 9:
                    symbol = company.get('Symbol', '').lower()
                    if symbol:
                        symbols.append(f'"{symbol}"')
        except Exception as e:
            print(f"Error parsing JSON: {e}")
    else:
        print(f"Request failed with status code {response.status_code}")

    return symbols

# Gom tất cả mã từ các trang
all_symbols = []
for skip in range(0, 2000, 20):
    all_symbols.extend(get_data_from_page(skip))

# In ra theo hàng ngang, ngăn cách bởi dấu phẩy
print(", ".join(all_symbols))

"a32", "aah", "aas", "abb", "abc", "abi", "abw", "ace", "acm", "acs", "acv", "afx", "ag1", "agf", "agp", "agx", "aic", "aig", "alv", "amd", "amp", "ams", "ant", "apc", "apf", "apl", "app", "apt", "art", "ata", "atb", "atg", "avc", "avg", "bal", "bbh", "bbm", "bbt", "bca", "bcb", "bcp", "bcr", "bcv", "bdg", "bdt", "bdw", "bel", "bge", "bgw", "bha", "bhc", "bhg", "bhi", "bhk", "bhp", "big", "bii", "bio", "blf", "bli", "bln", "blt", "bmd", "bmf", "bmg", "bmj", "bmk", "bmn", "bms", "bmv", "bnw", "bot", "bqb", "brr", "brs", "bsa", "bsd", "bsg", "bsh", "bsl", "bsp", "bsq", "bt1", "btb", "btd", "btg", "bth", "btn", "btu", "btv", "bvb", "bvg", "bvl", "bvn", "bwa", "bws", "c21", "c22", "c4g", "c92", "cab", "cad", "cat", "cbi", "cbs", "cc1", "cc4", "cca", "ccm", "ccp", "cct", "ccv", "cdg", "cdh", "cdo", "cdp", "cdr", "ce1", "ceg", "cen", "cfm", "cfv", "cgv", "ch5", "chc", "chs", "ci5", "cid", "cip", "ck8", "cka", "ckd", "clg", "clx", "cmd", "cmf", "cmi", "cmk", "cmm", "cmn", "cmp", "cmt", "cmw",

In [9]:
# def scrape_cafef_multiple_years(stock_code, report_type, year, url_suffix):
#     url = f"https://cafef.vn/du-lieu/bao-cao-tai-chinh/{stock_code}/{report_type}/{year}/{url_suffix}/bao-cao-tai-chinh-.chn"
#     headers = {"User-Agent": "Mozilla/5.0"}
#     print(f" Truy cập URL: {url}")
#     response = requests.get(url, headers=headers)

#     if response.status_code != 200:
#         print(f"Lỗi truy cập ({response.status_code}): {url}")
#         return None

#     soup = BeautifulSoup(response.text, "html.parser")
#     table = soup.find("table", id="tableContent")

#     if not table:
#         print("Không tìm thấy bảng dữ liệu.")
#         return None

#     df = pd.read_html(StringIO(str(table)))[0]
#     if isinstance(df.columns, pd.MultiIndex):
#         df.columns = df.columns.droplevel(0)

#     return df

def scrape_cafef_multiple_years(stock_code, report_type, year, url_suffix):
    url = f"https://cafef.vn/du-lieu/bao-cao-tai-chinh/{stock_code}/{report_type}/{year}/{url_suffix}/bao-cao-tai-chinh-.chn"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
        "Referer": "https://cafef.vn/"
    }

    print(f"- Truy cập URL: {url}")

    try:
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code != 200:
            print(f"--- Lỗi truy cập ({response.status_code}): {url}")
            return None

        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", id="tableContent")

        if not table:
            print("--- Không tìm thấy bảng dữ liệu.")
            return None

        df = pd.read_html(StringIO(str(table)))[0]
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(0)

        return df

    except requests.exceptions.Timeout:
        print(f"-------- Timeout (quá thời gian chờ): {url}")
        return None
    except Exception as e:
        print(f"-------- Lỗi không xác định khi truy cập {url}: {e}")
        return None

def convert_to_numeric(df):
    for col in df.columns[1:]:
        df[col] = df[col].apply(lambda x: str(x).replace(',', '').replace('(', '-').replace(')', '') if pd.notnull(x) else x)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def merge_financial_reports(stock_code, report_type):
    df2 = scrape_cafef_multiple_years(stock_code, report_type, 2024, "0/0/0")  # 2024 - 2021
    df1 = scrape_cafef_multiple_years(stock_code, report_type, 2020, "0/0/0")  # 2020 - 2019

    if df1 is None and df2 is None:
        print(">>>>>>>>> -- Không có dữ liệu nào được tải.")
        return None

    if df1 is not None and df2 is not None:
        merged_df = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
    else:
        merged_df = df1 if df1 is not None else df2

    # Đổi tên cột theo năm cố định
    new_columns = ["Chỉ tiêu", "2017", "2018", "2019", "2020", "TT1", "2021", "2022", "2023", "2024", "TT2"]
    if len(merged_df.columns) == len(new_columns):
        merged_df.columns = new_columns
    return merged_df.drop(columns=["TT1", "TT2"], errors="ignore")

def calculate_ratios(can_doi, kqkd):
    can_doi = convert_to_numeric(can_doi)
    kqkd = convert_to_numeric(kqkd)

    can_doi = can_doi.set_index("Chỉ tiêu")
    kqkd = kqkd.set_index("Chỉ tiêu")

    years = [str(y) for y in range(2017, 2025)]
    result = pd.DataFrame(index=range(1, 32), columns=["Chỉ tiêu"] + years)

    def get_value(df, key):
        matches = df.index[df.index.str.contains(key, case=False, na=False)]
        return df.loc[matches[0]] if len(matches) > 0 else pd.Series([np.nan]*len(years), index=years)

    # Lấy các dòng dữ liệu cần thiết
    ts_ngan = get_value(can_doi, "Tài sản ngắn hạn")
    no_ngan = get_value(can_doi, "Nợ ngắn hạn")
    ton_kho = get_value(can_doi, "Hàng tồn kho")
    phai_thu = get_value(can_doi, "Phải thu ngắn hạn")
    ts_cd = get_value(can_doi, "Tài sản cố định")
    tong_ts = get_value(can_doi, "TỔNG CỘNG TÀI SẢN")
    von_csh = get_value(can_doi, "Vốn chủ sở hữu")
    von_cp = get_value(can_doi, "Vốn góp của chủ sở hữu")
    tien = get_value(can_doi, "Tiền và các khoản tương đương tiền")
    no_dai = get_value(can_doi, "Nợ dài hạn")
    no_phai_tra = get_value(can_doi, "Nợ phải trả")

    dt = get_value(kqkd, "Doanh thu")
    lnst = get_value(kqkd, "Lợi nhuận sau thuế")
    lntt = get_value(kqkd, "Lợi nhuận thuần từ hoạt động kinh doanh")
    gvhb = get_value(kqkd, "Giá vốn")

    # Công thức tính toán chỉ số tài chính
    formulas = [
        ("Tài sản ngắn hạn / Nợ ngắn hạn", ts_ngan / no_ngan),
        ("(Tài sản ngắn – hàng tồn) / Nợ ngắn hạn", (ts_ngan - ton_kho) / no_ngan),
        ("Doanh thu / Tồn kho", dt / ton_kho),
        ("Phải thu / (Doanh thu/365)", phai_thu / (dt / 365)),
        ("Doanh thu / Tài sản cố định", dt / ts_cd),
        ("Doanh thu / Tổng tài sản", dt / tong_ts),
        ("Tổng nợ / Tổng tài sản", no_phai_tra / tong_ts),
        ("Vốn chủ sở hữu / Tổng tài sản", von_csh / tong_ts),
        ("Tổng nợ / Vốn cổ phần", no_phai_tra / von_cp),
        ("ROS (LNST/Doanh thu)", lnst / dt),
        ("ROA", lnst / tong_ts),
        ("ROE", lnst / von_csh),
        ("Tiền & tương đương / Nợ ngắn hạn", tien / no_ngan),
        ("(Tài sản ngắn – Nợ ngắn) / Tổng tài sản", (ts_ngan - no_ngan) / tong_ts),
        ("Phải thu ngắn hạn / Tài sản ngắn hạn", phai_thu / ts_ngan),
        ("Doanh thu / Vốn cổ phần", dt / von_cp),
        ("Nợ ngắn hạn / Tổng tài sản", no_ngan / tong_ts),
        ("Nợ dài hạn / Tổng tài sản", no_dai / tong_ts),
        ("Lợi nhuận trước thuế / Vốn cổ phần", lntt / von_cp),
        ("Tồn kho / Bán hàng", ton_kho / gvhb),
        ("Vòng quay tồn kho", gvhb / ton_kho),
        ("LNST / Doanh thu", lnst / dt),
        ("Giá vốn hàng bán / Phải thu", gvhb / phai_thu),
        ("TSCĐ / Tổng tài sản", ts_cd / tong_ts),
        ("Nợ ngắn hạn / Nợ phải trả", no_ngan / no_phai_tra),
        ("Nợ phải trả / Tổng tài sản", no_phai_tra / tong_ts),
        ("Log(Tổng tài sản)", np.log(tong_ts)),
        ("Vốn lưu động / Tổng tài sản", (ts_ngan - no_ngan) / tong_ts),
    ]

    for idx, (name, values) in enumerate(formulas, 1):
        result.loc[idx, "Chỉ tiêu"] = name
        result.loc[idx, years] = values.values if isinstance(values, pd.Series) else [np.nan] * len(years)

    # === TÍNH Z-SCORE VÀ PHÂN LOẠI RỦI RO ===
    von_luu_dong = ts_ngan - no_ngan
    EBIT = lntt
    x1 = von_luu_dong / tong_ts
    x2 = lnst / tong_ts
    x3 = EBIT / tong_ts
    x4 = von_csh / no_phai_tra
    x5 = dt / tong_ts

    z_score = 1.2 * x1 + 1.4 * x2 + 3.3 * x3 + 0.6 * x4 + 1.0 * x5
    risk_flag = z_score.apply(lambda z: 1 if z < 1.81 else 0)

    result.loc[30, "Chỉ tiêu"] = "Z-score"
    result.loc[30, years] = z_score.values

    result.loc[31, "Chỉ tiêu"] = "Rủi ro (1=rủi ro cao)"
    result.loc[31, years] = risk_flag.values

    return result

> Tên mã Công ty cần lấy

In [ ]:
# danh_sach_ma_cp = [] #insert company code

In [7]:
danh_sach_ma_cp = all_symbols

> Lấy dữ liệu

In [ ]:
df_candoi_all = pd.DataFrame()
df_kqkd_all = pd.DataFrame()
df_ratios_all = pd.DataFrame()

for ma_cp in danh_sach_ma_cp:
    try:
        print(f" > Đang xử lý mã: {ma_cp}...")
        can_doi = merge_financial_reports(ma_cp, "bsheet")
        kqkd = merge_financial_reports(ma_cp, "incsta")

        if can_doi is not None and kqkd is not None:
            ratios_df = calculate_ratios(can_doi, kqkd)

            # Thêm cột mã CP cho các bảng
            can_doi["ma_cp"] = ma_cp
            kqkd["ma_cp"] = ma_cp
            ratios_df["ma_cp"] = ma_cp

            df_candoi_all = pd.concat([df_candoi_all, can_doi], ignore_index=True)
            df_kqkd_all = pd.concat([df_kqkd_all, kqkd], ignore_index=True)
            df_ratios_all = pd.concat([df_ratios_all, ratios_df], ignore_index=True)
        else:
            print(f" <?> Không có dữ liệu cho mã: {ma_cp}")

    except Exception as e:
        print(f" ------- Lỗi khi xử lý mã {ma_cp}: {e}")
        continue
print("Done")

> chuyển đổi dữ liệu

In [ ]:
# Chuyển dạng từ wide -> long
value_vars = [col for col in df_ratios_all.columns if col.isnumeric()]
df_melted = df_ratios_all.melt(
    id_vars=["Chỉ tiêu", "ma_cp"],
    value_vars=value_vars,
    var_name="Năm",
    value_name="Giá trị"
)

# can_doi["ma_cp"] = ma_cp.upper()
# kqkd["ma_cp"] = ma_cp.upper()
# ratios_df["ma_cp"] = ma_cp.upper()
# Pivot thành dạng dọc
df_ratios_final = df_melted.pivot_table(
    index=["ma_cp", "Năm"],
    columns="Chỉ tiêu",
    values="Giá trị"
).reset_index()

# Sắp xếp đúng thứ tự chỉ tiêu
thu_tu_chi_tieu = [
    "Tài sản ngắn hạn / Nợ ngắn hạn",
    "(Tài sản ngắn – hàng tồn) / Nợ ngắn hạn",
    "Doanh thu / Tồn kho",
    "Phải thu / (Doanh thu/365)",
    "Doanh thu / Tài sản cố định",
    "Doanh thu / Tổng tài sản",
    "Tổng nợ / Tổng tài sản",
    "Vốn chủ sở hữu / Tổng tài sản",
    "Tổng nợ / Vốn cổ phần",
    "ROS (LNST/Doanh thu)",
    "ROA",
    "ROE",
    "Tiền & tương đương / Nợ ngắn hạn",
    "(Tài sản ngắn – Nợ ngắn) / Tổng tài sản",
    "Phải thu ngắn hạn / Tài sản ngắn hạn",
    "Doanh thu / Vốn cổ phần",
    "Nợ ngắn hạn / Tổng tài sản",
    "Nợ dài hạn / Tổng tài sản",
    "Lợi nhuận trước thuế / Vốn cổ phần",
    "Vốn hóa thị trường",
    "Tồn kho / Bán hàng",
    "Vòng quay tồn kho",
    "LNST / Doanh thu",
    "Giá vốn hàng bán / Phải thu",
    "TSCĐ / Tổng tài sản",
    "Nợ ngắn hạn / Nợ phải trả",
    "Nợ phải trả / Tổng tài sản",
    "Log(Tổng tài sản)",
    "Vốn lưu động / Tổng tài sản",
    "Z-score",
    "Rủi ro (1=rủi ro cao)"
]

existing_cols = [col for col in thu_tu_chi_tieu if col in df_ratios_final.columns]
df_ratios_final = df_ratios_final[["ma_cp", "Năm"] + existing_cols]

df_ratios_final["ma_cp"] = df_ratios_final["ma_cp"].str.upper()

In [ ]:
with pd.ExcelWriter("BCTT_tong_hop.xlsx") as writer:
    df_candoi_all.to_excel(writer, sheet_name="Bảng CDKT", index=False)
    df_kqkd_all.to_excel(writer, sheet_name="KQKD", index=False)
    df_ratios_final.to_excel(writer, sheet_name="Chỉ số tài chính", index=False)

print("Đã lưu file")

In [ ]:
from google.colab import files
files.download('bao_cao_tong_hop.xlsx')